Загружаем базу, с которой будем работать.

In [138]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data.csv')
melb_df = melb_data.copy()

Задание 1.1 Преобразуйте столбец Date в формат datetime и выделите квартал (quarter) продажи объектов недвижимости. Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.

In [139]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
second_quarter = melb_df['Date'].dt.quarter.value_counts().nlargest(2).values[1]
print('Second popular selling quarter:', second_quarter)

Second popular selling quarter: 4808


Задание 1.2 Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных category, исключив из преобразования столбцы Date, Rooms, Bedroom, Bathroom, Car.
В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных category.

In [140]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  object        
 1   Address        13580 non-null  object        
 2   Rooms          13580 non-null  int64         
 3   Type           13580 non-null  category      
 4   Price          13580 non-null  float64       
 5   Method         13580 non-null  category      
 6   SellerG        13580 non-null  object        
 7   Date           13580 non-null  datetime64[ns]
 8   Distance       13580 non-null  float64       
 9   Postcode       13580 non-null  float64       
 10  Bedroom2       13580 non-null  category      
 11  Bathroom       13580 non-null  float64       
 12  Car            13518 non-null  float64       
 13  Landsize       13580 non-null  float64       
 14  BuildingArea   7130 non-null   float64       
 15  YearBuilt      8205

None

Задание 2.2 Произведите сортировку столбца AreaRatio по убыванию. При этом индексы полученной таблицы замените на новые. Какое значение площади здания находится в строке 1558? Ответ округлите до целого числа.

In [141]:
melb_df.sort_values(by='AreaRatio', ascending=False, ignore_index=True)
print("Building area in string number 1558 = ", melb_df['BuildingArea'].values[1558])

KeyError: 'AreaRatio'

Задание 2.3 Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2. Отсортируйте полученную таблицу сначала по возрастанию числа комнат, а затем по убыванию средней площади комнат (MeanRoomsSquare). Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.

In [ ]:
melb_df[(melb_df['Type'] == 'townhouse')&(melb_df['Rooms'] > 2)].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index= True).loc[18,['Price']].values[0]

1300000.0

Задание 3.1 Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.

In [ ]:
melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False).index[0]

7

Задание 3.2 Какой регион имеет наименьшее стандартное отклонение по географической широте (Lattitude)?
В качестве ответа запишите название этого региона.

In [ ]:
melb_df.groupby('Regionname')['Lattitude'].std().sort_values().index[0]

'Western Victoria'

Задание 3.3 Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?
Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.
Не забудьте перевести даты в формат datetime.

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)

mask_date1 = pd.to_datetime('2017-05-01') < melb_df['Date']
mask_date2 = melb_df['Date'] < pd.to_datetime('2017-09-01')
melb_df[mask_date1&mask_date2].groupby('SellerG')['Price'].sum().sort_values().index[0]

'LITTLE'

Задание 4.2
Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms). Для какой комбинации признаков площадь здания наибольшая?
В качестве ответа запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.

In [ ]:
trarea = melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median',
    fill_value=0
)
display(trarea)


Rooms,1,2,3,4,5,6,7,8,10
Type,,,,,,,,,
house,126.0,126,126,141.0,177,126,216.5,126,126
townhouse,88.0,114,126,159.5,152,0,0.0,0,0
unit,69.5,110,126,126.0,0,171,0.0,126,0


Задание 4.3
Составьте сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости (Price) от риелторского агентства (SellerG) и типа здания (Type).
Во вновь созданной таблице найдите агентство, у которого медианная цена для зданий типа unit максимальна. В качестве ответа запишите название этого агентства.

In [ ]:
stprice = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
    fill_value=0
)
#display(stprice)
print(stprice[stprice['unit'].values == stprice['unit'].values.max()].index[0])

Nick


In [ ]:
ratings1 = pd.read_csv('data/ratings1.csv')
ratings2 = pd.read_csv('data/ratings2.csv')
dates = pd.read_csv('data/dates.csv')
movies = pd.read_csv('data/movies.csv')

Задание 5.2. Сколько уникальных фильмов представлено в таблице movies?

In [ ]:
print(movies['movieId'].nunique())

9742


Задание 5.3. Сколько уникальных пользователей в таблице ratings1?

In [ ]:
print(ratings1['userId'].nunique())

274


Задание 5.4. В каком году было выставлено больше всего оценок?
Для ответа на этот вопрос используйте таблицу dates.

In [ ]:
dates['date'] = pd.to_datetime(dates['date'])
print(dates['date'].dt.year.value_counts().nlargest(1).index[0])

2000


In [ ]:
df1 = pd.DataFrame({"Name": ["Pankaj", "Lisa"], "Surname": ["Sobolev", "Krasnova"]})
df2 = pd.DataFrame({"Role": ["Admin", "Editor"]})

display(df1)
display(df2)

,Name,Surname
0,Pankaj,Sobolev
1,Lisa,Krasnova


,Role
0,Admin
1,Editor


In [ ]:
df = pd.concat([df1,df2], axis=1)
display(df)

,Name,Surname,Role
0,Pankaj,Sobolev,Admin
1,Lisa,Krasnova,Editor


ЗАДАНИЕ 6.3 (ВЫПОЛНЯЕТСЯ В CODEBOARD НИЖЕ)
import pandas as pd
import os

def concat_users_files(path):
    """
    Вам необходимо написать функцию concat_user_files(path), параметром которой является path - путь до директории. 
    Функция должна объединить информацию из предоставленных вам файлов в один DataFrame и вернуть его. 
    Не забудьте обновить индексы результирующей таблицы после объединения.
    Учтите тот момент, что в результате объединения могут возникнуть дубликаты, от которых необходимо будет избавиться. 
    """
    l_csv = os.listdir(path)
    list_csv = sorted(l_csv)
    print(list_csv)
    
    i = 0
    while i < len(list_csv):
        #print(str(path + list_csv[i]))
        date_temp1 = pd.read_csv(str(path + list_csv[i]))
        print(date_temp1.head())
        if i == 0:
            date_temp2 = date_temp1.copy()
            i +=1
        else:
            date = pd.concat([date_temp2, date_temp1], ignore_index = True)
            date_temp2 = date.copy()
            i += 1
    date = date.drop_duplicates()
    print(date.head())
    return date
        


if __name__ == '__main__':
    data = concat_users_files('./Root/users/')

ЗАДАНИЕ 7.5 (ВЫПОЛНЯЕТСЯ В CODEBOARD НИЖЕ)

Даны две таблицы: items_df, в которой содержится информация о наличии товаров на складе, и purchase_df — с данными о покупках товаров.

items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})


purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})

Информация в таблицах представлена в виде следующих столбцов:

* item_id — идентификатор модели;
* vendor — производитель модели;
* stock_count — имеющееся на складе количество данных моделей (в штуках);
* purchase_id — идентификатор покупки;
* price — стоимость модели в покупке.

Сформируйте DataFrame merged, в котором в результате объединения purchase_df и items_df останутся модели, которые учтены на складе и имели продажи.
Из таблицы merged найдите суммарную выручку, которую можно было бы получить от продажи всех товаров, которые учтены на складе и имели продажи. Результат занесите в переменную income.

In [ ]:
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})
purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})

merged = items_df.merge(purchase_df, how='inner', on='item_id')
income = (merged['stock_count']*merged['price']).sum()
print('Income:', income)

Shape item_id: 10, shape purchase_id: 10, shape merged: 9


,item_id,vendor,stock_count,purchase_id,price
0,417283,Samsung,54,101,13900
1,849734,LG,33,101,5330
2,132223,Apple,122,101,38200
3,573943,Apple,18,112,49990
4,19475,LG,102,121,9890
5,3294095,Apple,43,145,33000
6,382043,Samsung,77,145,67500
7,302948,Samsung,143,145,34500
8,100132,LG,60,221,11400


Income: 19729490


## Итоговые задания модуля

In [ ]:
import pandas as pd

ratings_movies = pd.read_csv('data/ratings_movies.csv', sep=',', index_col='Unnamed: 0')
display(ratings_movies.head())

#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
    

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Задание 8.1
Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
У скольких фильмов не указан год их выпуска?

In [ ]:
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
print(ratings_movies['year_release'].shape[0] - ratings_movies['year_release'].count())

18


Задание 8.2
Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
В качестве ответа запишите название этого фильма без указания года его выпуска.

In [ ]:
mean_rating_1999 = ratings_movies[ratings_movies['year_release'] == 1999.0].groupby('movieId')['rating'].mean()
result_movieId_1999 = mean_rating_1999.nsmallest(1).index[0]
print(ratings_movies[ratings_movies['movieId'] == result_movieId_1999]['title'].values[0])


Bloodsport: The Dark Kumite (1999)


Задание 8.3
Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [ ]:
print(ratings_movies[ratings_movies['year_release'] == 2010.0].groupby('genres')['rating'].mean().nsmallest(1).index[0])


Action|Sci-Fi


Задание 8.4
Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов? В качестве ответа запишите идентификатор этого пользователя.

In [ ]:
print(ratings_movies.groupby('userId')['genres'].agg('nunique').nlargest(1).index[0])

599


Задание 8.5
Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
В качестве ответа укажите идентификатор этого пользователя.
Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), можно воспользоваться методом agg() на сгруппированных данных.

In [ ]:
print(ratings_movies.groupby('userId')['rating'].agg(['count','mean']).sort_values(by='count').sort_values(by='mean',ascending=False).index[0])

53


Задание 8.6
Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [ ]:
mask2018 = ratings_movies['year_release'] == 2018.0
res_genres = ratings_movies[mask2018].groupby('genres')['rating'].agg(['mean','count'])
result_genres = res_genres[res_genres['count'] > 10]['mean'].nlargest(1).index[0]
print(result_genres)


Action|Adventure|Sci-Fi


Задание 8.7
Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра. Выберите верные варианты ответа, исходя из построенной таблицы:

In [170]:
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])
ratings_movies['year_rating'] = ratings_movies['date'].dt.year
pivot_ryg = ratings_movies.pivot_table(values='rating',index='year_rating', columns='genres', aggfunc='mean')
#display(pivot_ryg)
print(pivot_ryg['Action|Adventure'].min() <= 3.0)
print(pivot_ryg['Action|Adventure|Animation|Children|Comedy|IMAX'].nlargest(1).index[0] == 2010)
print(pivot_ryg['Animation|Children|Mystery'].loc[2018])
print(pivot_ryg['Comedy'].nlargest().sort_index())

False
False
5.0
year_rating
1997    3.409091
1999    3.606061
2008    3.426667
2012    3.478477
2013    3.489474
Name: Comedy, dtype: float64


Дано два файла:

* orders.csv, содержащий данные о заказах;
* products.csv, содержащий данные о товарах.

Скачать файлы можно здесь (оба файла находятся в едином zip-архиве — распакуйте его, прежде чем продолжать работу!)

Прочитайте данные файлы, посмотрите на содержимое таблиц и проанализируйте его.

Подумайте, как связаны данные таблицы (какими ключевыми столбцами).

In [181]:
import pandas as pd

orders = pd.read_csv('data/orders.csv', sep=';')
display(orders.head())
display(orders.shape)

products = pd.read_csv('data/products.csv', sep=';')
display(products.head())
display(products.shape)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5


(18, 9)

,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR


(14, 4)

Объедините заданные таблицы в таблицу orders_products, чтобы в результирующей таблице оказалась информация обо всех заказах, но не оказалось информации о продуктах, на которых заказов ещё не поступало. 

In [183]:
orders_products = orders.merge(products, how='left', left_on='ID товара', right_on='Product_ID', )
display(orders_products)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,91.0,"Носки Честные, муж",50.0,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,103.0,"Носки Подарочные, муж",199.0,RUR
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,104.0,"Носки Подарочные, жен",249.0,RUR


Задание 8.10
Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
В ответ запишите идентификатор этого покупателя (ID Покупателя).
Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара, умноженное на его цену.

In [189]:
mask = orders_products['Оплачен'] == 'Да'
orders_products['Income'] = orders_products[mask]['Количество']*orders_products[mask]['Price']
print(orders_products.groupby(by='ID Покупателя')['Income'].agg('sum'))

ID Покупателя
1         0.0
5     13043.0
7     17096.0
8      1344.0
9         0.0
10        0.0
Name: Income, dtype: float64
